# CS170: Zoom Breakout Rooms
Input: n = # of students,  
Smax = Stress budget,  
h_i_j, s_i_j = happiness & stress between each student pair  

Output:  
H_r = Happiness of breakout room r  
S_r = stress of room r  
H_total = total happiness  
k = # of rooms opened  

Constraints:  
S_r < Smax / k  
k = # of rooms opened.   

Goal: maximize H total.

## importing files & libs

In [1]:
!ls

README.md                    inputs_outputs
__pycache__                  log-10-200.txt
breakout-rooms-8a27e55.ipynb log-med-1-50.txt
breakout-rooms.ipynb         log.txt
cp_outputs                   outputs
cp_solver.py                 parse.py
input_create.ipynb           reflection.txt
input_create.ipynb.txt       samples
input_create.py              solver.py
inputs                       test_write.py
inputs.zip                   utils.py


In [2]:
import utils
import utils as ut
import parse as ps
import networkx as nx
from docplex.mp.model import Model
import numpy as np
import pandas

## Parsing Variables
Input: n = # of students,
Smax = Stress budget,
h_i_j, s_i_j = happiness & stress between each student pair

In [3]:
def process_input(filename):
    G, S_MAX = ps.read_input_file(filename)
    N = len(G)
    D = nx.to_dict_of_dicts(G)
    R = range(N)
    S_mat = np.zeros((N,N))
    H_mat = np.zeros((N,N))
    dic = D.copy()
    idx = []
    for i in R:
        dic.pop(i);
        idx.extend([(i, i, k) for k in R])
        for j in dic.keys():
            info = D[i][j]
            S_mat[i,j]=info['stress']
            H_mat[i,j]=info['happiness']
            idx.extend([(i, j, k) for k in R])
    #         D[i][j]['h'] = D[i][j].pop('happiness')
    #         D[i][j]['s'] = D[i][j].pop('stress')
    #         print("{}-{}: h={}, s={}".format(i, j, info['happiness'], info['stress']))
#     print(S_mat)
#     print(H_mat)
    print(len(idx))
    return G, S_MAX, N, R, S_mat, H_mat, idx


# Build model (MP)

In [4]:
G, S_MAX, N, R, S_mat, H_mat, idx = process_input("inputs_outputs/10.in")
msol = 0
model = 0
x = 0

550


In [5]:
def init_model(N):
# ref to the mp sudoku example: https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html
    global model, x
    model = Model("breakout_room_mp")
    R = range(N)
    # X is room assignment; key is (i, j, r) where i,j is a pair of students, and r is the room they are assigned to
    x = model.binary_var_dict(idx, name="X") 
    # room_count = model.integer_var(1, N, "room_count")

    # constraint 1: each pair can only be assigned to one room. 
    # -> for each pair, sum of all different room choices == 1
    for i in R:
    #     model.add_constraint(model.sum(x[j,i,k] for j in range(i) for k in R)
    #                     +    model.sum(x[i,j,k] for j in range(i,N) for k in R) == 1)
        for j in range(i, N):
            pair_sum = model.sum(x[i,j,k] for k in R)
            model.add_constraint(pair_sum <= 1)

    # constraint *2: generate "clique" for people appeared in a pair in a room. using triangle/rectangle, in each k layer.
    # constraint 3: everyone can only be assigned to one room.
    # -> other room sum of pairs including someone assigned == 0
    for k in R:
        for a in R: 
            for b in range(a + 1,N):
                model.add_constraint(model.indicator_constraint(
                    x[a,b,k], 
                    model.sum(x[a,i,r] for i in range(a+1,N) for r in R if r != k)+ 
                    model.sum(x[i,a,r] for i in range(a) for r in R if r != k)
                    ==0))
                model.add_constraint(model.indicator_constraint(
                    x[a,b,k], 
                    model.sum(x[b,i,r] for i in range(b+1,N) for r in R if r != k)+
                    model.sum(x[i,b,r] for i in range(b) for r in R if r != k)
                    == 0))
                for c in range(b + 1,N):
                    model.add_constraint(x[a,b,k] + x[a,c,k] + x[b,c,k] != 2)
                    for d in range(c + 1,N):
                        model.add_constraint(model.if_then((x[a,b,k] + x[c,d,k]) == 2, x[a,c,k] == 1))
    return model, x

    # constraint 2: rooms are using consecutive numbers starting from 0. 
    # if nobody is in one room, do not assign people to rooms with no > this one
    # also: calculate room count
    # now: count all partners in the assigned group!
    # for k in range(N-1):
    # #     room_sum = model.sum(x[i,j,k] for i in R for j in range(i,N))
    #     for i in R:
    #         for j in range(i, N):
    #             model.add_constraint(model.if_then(room_sum == 0, x[i,j,k+1]==0))
    #             model.add_constraint(model.if_then(room_sum == 0, room_count <= k))
    #             model.add_constraint(model.if_then(room_sum != 0, room_count >= k + 1))

In [6]:
def model_and_solve(NBROOMS):
# ref to the mp sudoku example: https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html
    global model
        # constraint 3: stress level of each room is under the limit. 
    # how to find the number of rooms assigned? <- create a new variable
    constraints = []
    for r in R:
        S_room = model.sum(S_mat[i,j] * x[i,j,r] for i in R for j in range(i,N))
        constraints.append(model.add_constraint(S_room <= S_MAX / NBROOMS))
    # goal: maximize happiness
    model.maximize(model.sum(H_mat[i,j] * x[i,j,k] for i in R for j in range(i,N) for k in R))
    model.set_time_limit(5)
    msol = model.solve()
    model.remove_constraints(constraints)
    model.remove_objective()
    print(f"happiness: {msol.objective_value}")
#     print(msol)
    return msol

In [7]:
def clean_model_solution(msol, x, N):
    msol_dict = msol.get_value_dict(x)
    msol_np = np.concatenate((list(msol_dict.keys()), np.array([list(msol_dict.values())]).T), axis=1).astype(int)
    msol_nz = msol_np[msol_np[:,3].nonzero()]
    msol_rooms = msol_nz[:,2]
    msol_unique_rooms = np.unique(msol_rooms)
    msol_unique_rooms
    number_of_rooms = len(msol_unique_rooms)
    for i in range(number_of_rooms):
        msol_rooms = np.where(msol_rooms==msol_unique_rooms[i],i,msol_rooms)
    msol_rooms
    msol_nz[:,2]=msol_rooms
    for i in np.setdiff1d(np.arange(N), np.unique(np.concatenate([msol_nz[:,0],msol_nz[:,1]]))):
        msol_nz = np.append(msol_nz, [[i,i, number_of_rooms,1]],axis=0)
        msol_rooms = np.append(msol_rooms, number_of_rooms)
        number_of_rooms += 1
    print(msol_nz, number_of_rooms)
    return msol_nz, msol_rooms, number_of_rooms
    
def is_valid(msol_nz, number_of_rooms):
    S_room_max = S_MAX/number_of_rooms
    for i in range(number_of_rooms):
        room = msol_nz[np.argwhere(msol_nz[:,2] == i).flatten()]
        S_r = S_mat[room[:,0], room[:,1]].sum()
        if (S_r > S_room_max):
            print(f"room {i} stress = {S_r}, exceeded {S_room_max}")
            return False
    return True

def to_dict(msol_nz, msol_rooms, number_of_rooms):
    sol_dict = {}
    for i in range(number_of_rooms):
        sel = msol_nz[np.argwhere(msol_rooms == i).flatten()]
        sol_dict.update(dict(zip(sel[:,0], sel[:,2])))
        sol_dict.update(dict(zip(sel[:,1], sel[:,2])))
    return sol_dict

In [8]:
# processes to get valid solution
def solve_process(input_path):
    global G, S_MAX, N, R, S_mat, H_mat, idx, msol, x
    G, S_MAX, N, R, S_mat, H_mat, idx = process_input(input_path)
#     model, x = init_model(N)
    nbrooms = 1
    while(nbrooms < N):
        msol = model_and_solve(nbrooms)
        sol_nz, msol_rooms, number_of_rooms = clean_model_solution(msol, x, N)
        if (number_of_rooms != N and is_valid(sol_nz, number_of_rooms)):
            print(f"success! nbrooms = {nbrooms}")
            return to_dict(sol_nz, msol_rooms, number_of_rooms)
        print(f"nbrooms = {nbrooms} failed. Trying {nbrooms + 1}")
        nbrooms += 1
    print("FAILED to find good solution")
    return {}
# sol_dict, number_of_rooms, sol_nz = solve_process("inputs_outputs/20.in")

# Solve small inputs

In [9]:
init_model(10)

(<docplex.mp.model.Model at 0x7f9bd30e2d90>,
 {(0, 0, 0): docplex.mp.Var(type=B,name='X_0_0_0'),
  (0, 0, 1): docplex.mp.Var(type=B,name='X_0_0_1'),
  (0, 0, 2): docplex.mp.Var(type=B,name='X_0_0_2'),
  (0, 0, 3): docplex.mp.Var(type=B,name='X_0_0_3'),
  (0, 0, 4): docplex.mp.Var(type=B,name='X_0_0_4'),
  (0, 0, 5): docplex.mp.Var(type=B,name='X_0_0_5'),
  (0, 0, 6): docplex.mp.Var(type=B,name='X_0_0_6'),
  (0, 0, 7): docplex.mp.Var(type=B,name='X_0_0_7'),
  (0, 0, 8): docplex.mp.Var(type=B,name='X_0_0_8'),
  (0, 0, 9): docplex.mp.Var(type=B,name='X_0_0_9'),
  (0, 1, 0): docplex.mp.Var(type=B,name='X_0_1_0'),
  (0, 1, 1): docplex.mp.Var(type=B,name='X_0_1_1'),
  (0, 1, 2): docplex.mp.Var(type=B,name='X_0_1_2'),
  (0, 1, 3): docplex.mp.Var(type=B,name='X_0_1_3'),
  (0, 1, 4): docplex.mp.Var(type=B,name='X_0_1_4'),
  (0, 1, 5): docplex.mp.Var(type=B,name='X_0_1_5'),
  (0, 1, 6): docplex.mp.Var(type=B,name='X_0_1_6'),
  (0, 1, 7): docplex.mp.Var(type=B,name='X_0_1_7'),
  (0, 1, 8): docple

In [18]:
with open('log.txt', 'a') as f:
    original_stdout = sys.stdout
    sys.stdout = f # Change the standard output to the file we created.
    print('LOG STARTS')
    for i in range(10,200):
        print(f"\n--------------\nCURRENT i = {i}\n")
        try:
            sol_dict = solve_process(f"inputs/small-{i}.in")
            ps.write_output_file(sol_dict, f"outputs/small-{i}.out")
        except: # catch *all* exceptions
            e = sys.exc_info()[0]
            print(f"ERROR OCCURED: i = {i}, {repr(e)}")
            continue
    sys.stdout = original_stdout # Reset the standard output to its original value

## Debug

TODO: 
1. Don't rebuild constraints every time. Instead, just remove the S_mat constraint, remove objective, and add new ones back.
2. See why the rectangle constraint does not work. 

In [10]:
solve_process("inputs/small-10.in")

550


DOcplexLimitsExceeded: **** Promotional version. Problem size limits exceeded, CPLEX code=1016

In [41]:
10 * 9 / 2

45.0

In [42]:
45 * 43

1935

In [43]:
R = range(10)
count = 0
for a in R: 
    for b in range(a + 1,N):
        for c in range(b + 1,N):
            for d in range(c + 1,N):
                count +=1
                print(f"(x[{a},{b},k] + x[{c},{d},k]) == 2, x[{a},{c},k] == 1)")
print(count)

(x[0,1,k] + x[2,3,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[2,4,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[2,5,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[2,6,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[2,7,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[2,8,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[2,9,k]) == 2, x[0,2,k] == 1)
(x[0,1,k] + x[3,4,k]) == 2, x[0,3,k] == 1)
(x[0,1,k] + x[3,5,k]) == 2, x[0,3,k] == 1)
(x[0,1,k] + x[3,6,k]) == 2, x[0,3,k] == 1)
(x[0,1,k] + x[3,7,k]) == 2, x[0,3,k] == 1)
(x[0,1,k] + x[3,8,k]) == 2, x[0,3,k] == 1)
(x[0,1,k] + x[3,9,k]) == 2, x[0,3,k] == 1)
(x[0,1,k] + x[4,5,k]) == 2, x[0,4,k] == 1)
(x[0,1,k] + x[4,6,k]) == 2, x[0,4,k] == 1)
(x[0,1,k] + x[4,7,k]) == 2, x[0,4,k] == 1)
(x[0,1,k] + x[4,8,k]) == 2, x[0,4,k] == 1)
(x[0,1,k] + x[4,9,k]) == 2, x[0,4,k] == 1)
(x[0,1,k] + x[5,6,k]) == 2, x[0,5,k] == 1)
(x[0,1,k] + x[5,7,k]) == 2, x[0,5,k] == 1)
(x[0,1,k] + x[5,8,k]) == 2, x[0,5,k] == 1)
(x[0,1,k] + x[5,9,k]) == 2, x[0,5,k] == 1)
(x[0,1,k] + x[6,7,k]) == 2, x[0,6,k] == 1)
(x[0,1,k] +

In [37]:
solve_process("inputs/small-8.in")

550
happiness: 0.0
[[0 0 0 1]
 [1 1 1 1]
 [2 2 2 1]
 [3 3 3 1]
 [4 4 4 1]
 [5 5 5 1]
 [6 6 6 1]
 [7 7 7 1]
 [8 8 8 1]
 [9 9 9 1]] 10
nbrooms = 1 failed. Trying 2
happiness: 330.231
[[0 7 1 1]
 [0 8 1 1]
 [1 5 3 1]
 [2 3 0 1]
 [4 9 2 1]
 [7 8 1 1]
 [6 6 4 1]] 5
room 0 stress = 2.913, exceeded 2.7920000000000003
nbrooms = 2 failed. Trying 3
happiness: 301.067
[[0 6 1 1]
 [0 9 1 1]
 [1 2 2 1]
 [3 4 0 1]
 [6 9 1 1]
 [7 8 3 1]
 [5 5 4 1]] 5
room 2 stress = 4.614, exceeded 2.7920000000000003
nbrooms = 3 failed. Trying 4
happiness: 260.68899999999996
[[0 8 1 1]
 [0 9 1 1]
 [1 5 0 1]
 [2 3 2 1]
 [4 7 3 1]
 [8 9 1 1]
 [6 6 4 1]] 5
room 2 stress = 2.913, exceeded 2.7920000000000003
nbrooms = 4 failed. Trying 5
happiness: 261.802
[[0 6 0 1]
 [1 5 0 1]
 [2 9 3 1]
 [3 4 2 1]
 [7 8 1 1]] 4
success! nbrooms = 5


{0: 0, 1: 0, 6: 0, 5: 0, 7: 1, 8: 1, 3: 2, 4: 2, 2: 3, 9: 3}

In [16]:
G, S_MAX = ps.read_input_file("inputs/small-7.in")
ps.read_output_file("outputs/small-7.out", G, S_MAX)

# of rooms:  4
D:  {0: 0, 7: 0, 8: 0, 1: 1, 6: 1, 9: 1, 2: 2, 3: 2, 4: 3, 5: 3}


{0: 0, 7: 0, 8: 0, 1: 1, 6: 1, 9: 1, 2: 2, 3: 2, 4: 3, 5: 3}

In [15]:
G, S_MAX = ps.read_input_file("inputs/small-8.in")
ps.read_output_file("outputs/small-8.out", G, S_MAX)

# of rooms:  4
D:  {1: 0, 2: 0, 5: 0, 9: 0, 3: 1, 4: 1, 7: 2, 8: 2, 0: 3, 6: 3}
too stressful!  20.098


AssertionError: 

In [17]:
G, S_MAX = ps.read_input_file("inputs/small-9.in")
ps.read_output_file("outputs/small-9.out", G, S_MAX)

# of rooms:  2
D:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 6: 0, 5: 1, 7: 1, 8: 1, 9: 1}


{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 6: 0, 5: 1, 7: 1, 8: 1, 9: 1}

## Cleaning Model Attempt

In [197]:
msol_dict = msol.get_value_dict(x)
msol_np = np.concatenate((list(msol_dict.keys()), np.array([list(msol_dict.values())]).T), axis=1).astype(int)
msol_nz = msol_np[msol_np[:,3].nonzero()]
msol_nz

array([[0, 4, 2, 1],
       [0, 5, 2, 1],
       [1, 2, 5, 1],
       [1, 3, 5, 1],
       [2, 3, 5, 1],
       [4, 5, 2, 1],
       [6, 8, 1, 1],
       [7, 9, 6, 1]])

In [198]:
msol_rooms = msol_nz[:,2]
msol_unique_rooms = np.unique(msol_rooms)
msol_unique_rooms
number_of_rooms = len(msol_unique_rooms)

In [199]:
for i in range(number_of_rooms):
    msol_rooms = np.where(msol_rooms==msol_unique_rooms[i],i,msol_rooms)
msol_rooms
msol_nz[:,2]=msol_rooms
msol_nz

array([[0, 4, 1, 1],
       [0, 5, 1, 1],
       [1, 2, 2, 1],
       [1, 3, 2, 1],
       [2, 3, 2, 1],
       [4, 5, 1, 1],
       [6, 8, 0, 1],
       [7, 9, 3, 1]])

In [200]:
for i in np.setdiff1d(np.arange(10), np.unique(np.concatenate([msol_nz[:,0],msol_nz[:,1]]))):
    msol_nz = np.append(msol_nz, [[i,i, number_of_rooms,1]],axis=0)
    msol_rooms = np.append(msol_rooms, number_of_rooms)
    number_of_rooms += 1
print(msol_nz, number_of_rooms, msol_rooms)

[[0 4 1 1]
 [0 5 1 1]
 [1 2 2 1]
 [1 3 2 1]
 [2 3 2 1]
 [4 5 1 1]
 [6 8 0 1]
 [7 9 3 1]] 4 [1 1 2 2 2 1 0 3]


In [212]:
is_valid(msol_nz)

room 1 stress = 32.256, exceeded 24.21025


False

In [204]:
sol_dict = {}
for i in range(number_of_rooms):
    sel = msol_nz[np.argwhere(msol_rooms == i).flatten()]
    sol_dict.update(dict(zip(sel[:,0], sel[:,2])))
    sol_dict.update(dict(zip(sel[:,1], sel[:,2])))
sol_dict

{6: 0, 8: 0, 0: 1, 4: 1, 5: 1, 1: 2, 2: 2, 3: 2, 7: 3, 9: 3}

In [205]:
ut.is_valid_solution(sol_dict,G,S_MAX, number_of_rooms)

too stressful!  32.256


False

In [174]:
S_MAX / 4

24.21025

## Build Model (~~CP~~) (MP)

For the actual CP model, check `cp_optimize` notebook.

Thought this would be solved with CP, accidentally created the model with MP but still kind of got it to work. Seems like CP still has better performance in comparison, although this MP model is much faster than the previous one and have better outputs satisfying the constriants (previous MP has many bad outputs that did not satisfy constraint).

Use the 2D idea to model. Since CP doesn't require convex, multiplying variables will be allowed and it will be easy to model in S_mat and H_mat. 

Biggest issue is the room_count variable still can't be used here, due to exceeding quadratic limits when considering stress constriant and happiness objective. CP allows it and solves the problem, and if using this model, looping through all possible room_counts or strategically select them as in `solve process` works for medium input at least.

variable: use binary_var_dict, based on each (i,k) where i is student, k is room being assigned too.;
room count variable

constriants: 

1. 1 student can assigned to 1 and only 1 room. 
2. room_count variable, force rooms with smaller # to be filled first and use that to get the range of room_count
3. (Per input): `sum(S_mat[i][j] * v[i][k] * v[j][k] for i in ... j in ...) <= S_MAX / roomcount`
Not sure if the roomcount will make the problem too slow to solve. If it does, take out constraint 2 and loop through possible room count variables. (Not very practical when N = 50)

Goal: 
maximize `sum(H_mat[i][j] * v[i][k] * v[j][k] for i in ... j in ... k in ...)`

In [4]:
import docplex.cp.model as cp
# Create model
# Initialize model variable sets
# total_rooms = 0

In [5]:
def clean_solution(cpsol):
    cpsol_dict = cpsol.get_value_dict(v, False)
    cpsol_np = np.concatenate((list(cpsol_dict.keys()), np.array([list(cpsol_dict.values())]).T), axis=1).astype(int)
    cpsol_rooms = cpsol_np[:,1]
    cpsol_unique_rooms = np.unique(cpsol_rooms)
    number_of_rooms = len(cpsol_unique_rooms)
    for i in range(number_of_rooms):
        cpsol_rooms = np.where(cpsol_rooms==cpsol_unique_rooms[i],i,cpsol_rooms)
    cpsol_rooms
    cpsol_np[:,1]=cpsol_rooms
#     print(cpsol_np)
    cpsol_dict = {}
    cpsol_dict.update(dict(zip(cpsol_np[:,0], cpsol_np[:,1])))
    return cpsol_dict, number_of_rooms

def is_valid(cpsol_dict, number_of_rooms):
    return ut.is_valid_solution(cpsol_dict, G, S_MAX, number_of_rooms)

In [16]:
def cp_model_and_solve(N, NBROOMS): # N = # of students
    # ref to the mp sudoku example: https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_mdl.html
    global mdl, v
#     mdl = cp.CpoModel("breakout_room_cp")
    mdl = Model('breakout_room_mp')
    R = range(N)
    idx = [(i,k) for i in R for k in R]
    # v is room assignment; key is (i, j, r) where i,j is a pair of students, and r is the room they are assigned to
    v = mdl.binary_var_dict(idx, name="v") 
#     room_count = mdl.integer_var(1, N, "room_count")

    # constraint 1: everyone is assigned to one room. 
    for i in R:
            mdl.add(mdl.sum(v[i,k] for k in R) == 1)
    
#     mdl.add(cp.sum(cp.sum(v[i,k] for i in R) != 0 for k in R) == room_count)

    constraints = []
    for r in R:
        S_room = cp.sum(S_mat[i,j] * v[i,r] * v[j,r] for i in R for j in range(i,N))
        constraints.append(mdl.add(S_room <= S_MAX / NBROOMS))
    # goal: maximize happiness
    mdl.maximize(cp.sum(H_mat[i,j] * v[i,k] * v[j,k] for i in R for j in range(i,N) for k in R))
    cpsol = mdl.solve(TimeLimit=20)
    return cpsol

In [7]:
G, S_MAX, N, R, S_mat, H_mat, idx = process_input("inputs/medium-1.in")

4200


In [8]:
def solve_process(input_path):
    global G, S_MAX, N, R, S_mat, H_mat, idx
    G, S_MAX, N, R, S_mat, H_mat, idx = process_input(input_path)
#     model, x = init_model(N)
    nbrooms = 2
    while nbrooms < N:
        sol = cp_model_and_solve(N, nbrooms)
        sol_dict, actual_nbrooms = clean_solution(sol)
        if (is_valid(sol_dict, actual_nbrooms)):
            print(f"success! nbrooms = {nbrooms}")
            print(sol_dict, actual_nbrooms)
            return sol_dict
        print(f"nbrooms = {nbrooms} failed.")
        new_nbrooms = np.round((nbrooms + actual_nbrooms) / 2)
        nbrooms = new_nbrooms if new_nbrooms > nbrooms else nbrooms + 1
        print(f"trying {nbrooms}")
    print("FAILED to find good solution")
    raise Exception('Could not find viable solution') 

In [9]:
from contextlib import redirect_stdout
import sys

with open('log-med-1-50.txt', 'a+') as f:
    with redirect_stdout(f):
        print('MEDIUM LOG STARTS')
        for i in range(1,50):
            print(f"\n--------------\nCURRENT i = {i}\n")
            sys.stdout.flush()
            try:
                sol_dict = solve_process(f"inputs/medium-{i}.in")
                sys.stdout.flush()
                ps.write_output_file(sol_dict, f"cp_outputs/medium-{i}.out")
                sys.stdout.flush()
            except: # catch *all* exceptions
                e = sys.exc_info()[0]
                print(f"ERROR OCCURED: i = {i}, {repr(e)}")
                sys.stdout.flush()
                continue

In [213]:
mdl.get_solve_details()

docplex.mp.SolveDetails(time=20.0116,status='time limit exceeded')

In [17]:
solve_process("inputs/medium-1.in")

4200


CpoException: Can not execute command 'cpoptimizer -angel'. Please check availability of required executable file.

In [212]:
msol_dict = ps.read_output_file("outputs/small-133.out", G, S_MAX)
ut.calculate_happiness(msol_dict, G)

# of rooms:  2
D:  {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}


AssertionError: 

# Build Model (MP, based on vertex not edges, 2D)

Use 2D model. However, this requires multiplying x\[i,k] and x\[j,k]; current version of docplex doesn't have this (quad constraint/ quad goal).  
Community edition has this feature, but # of variables and constraints are limited < 1000 (won't work for N=50 since vars = 2500).  
variable dict: (i, k) where i is student, k is room assigned

In [ ]:
cp.

In [6]:
def init_mdl(N):
# ref to the mp sudoku example: https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_mdl.html
    global mdl, v
    mdl = cp.Cp("breakout_room_mp")
    R = range(N)
    idx = [(i,k) for i in R for k in R]
    # v is room assignment; key is (i, j, r) where i,j is a pair of students, and r is the room they are assigned to
    v = mdl.binary_var_dict(idx, name="v") 
    room_count = mdl.integer_var(1, N, "room_count")

    # constraint 1: everyone is assigned to one room. 
    for i in R:
            mdl.add_constraint(mdl.sum(v[i,k] for k in R) == 1)

    return mdl, v

    # constraint 2: rooms are using consecutive numbers starting from 0. 
    # if nobody is in one room, do not assign people to rooms with no > this one
    # also: calculate room count
    # now: count all partners in the assigned group!
    for k in range(N-1):
        room_sum = mdl.sum(v[i,k] for i in R)
        for i in R:
            mdl.add_constraint(mdl.if_then(room_sum == 0, v[i,k+1]==0))
            mdl.add_constraint(mdl.if_then(room_sum == 0, room_count <= k))
            mdl.add_constraint(mdl.if_then(room_sum != 0, room_count >= k + 1))

In [7]:
init_mdl(10)

(<docplex.mp.model.Model at 0x7fb9537420d0>,
 {(0, 0): docplex.mp.Var(type=B,name='v_0_0'),
  (0, 1): docplex.mp.Var(type=B,name='v_0_1'),
  (0, 2): docplex.mp.Var(type=B,name='v_0_2'),
  (0, 3): docplex.mp.Var(type=B,name='v_0_3'),
  (0, 4): docplex.mp.Var(type=B,name='v_0_4'),
  (0, 5): docplex.mp.Var(type=B,name='v_0_5'),
  (0, 6): docplex.mp.Var(type=B,name='v_0_6'),
  (0, 7): docplex.mp.Var(type=B,name='v_0_7'),
  (0, 8): docplex.mp.Var(type=B,name='v_0_8'),
  (0, 9): docplex.mp.Var(type=B,name='v_0_9'),
  (1, 0): docplex.mp.Var(type=B,name='v_1_0'),
  (1, 1): docplex.mp.Var(type=B,name='v_1_1'),
  (1, 2): docplex.mp.Var(type=B,name='v_1_2'),
  (1, 3): docplex.mp.Var(type=B,name='v_1_3'),
  (1, 4): docplex.mp.Var(type=B,name='v_1_4'),
  (1, 5): docplex.mp.Var(type=B,name='v_1_5'),
  (1, 6): docplex.mp.Var(type=B,name='v_1_6'),
  (1, 7): docplex.mp.Var(type=B,name='v_1_7'),
  (1, 8): docplex.mp.Var(type=B,name='v_1_8'),
  (1, 9): docplex.mp.Var(type=B,name='v_1_9'),
  (2, 0): docpl

In [28]:
def model_and_solve(NBROOMS):
# ref to the mp sudoku example: https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html
    global model
        # constraint 3: stress level of each room is under the limit. 
    # how to find the number of rooms assigned? <- create a new variable
    constraints = []
    for r in R:
        S_room = model.sum(S_mat[i,j] * x[i,r] * x[j,r] for i in R for j in range(i,N))
        constraints.append(model.add_constraint(S_room <= S_MAX / NBROOMS))
    # goal: maximize happiness
    model.maximize(model.sum(H_mat[i,j] * x[i,j,k] for i in R for j in range(i,N) for k in R))
    model.set_time_limit(5)
    msol = model.solve()
    model.remove_constraints(constraints)
    model.remove_objective()
    print(f"happiness: {msol.objective_value}")
#     print(msol)
    return msol

In [1]:
!cpoptimizer

/bin/sh: cpoptimizer: command not found


In [ ]:
G, S_MAX, N, R, S_mat, H_mat, idx_3 = process_input("inputs_outputs/10.in")